In [9]:
import requests
import time
import random
import json
import logging as l

In [2]:
url = "https://www.traderjoes.com/api/graphql"
query = "query SearchProducts($categoryId: String, $currentPage: Int, $pageSize: Int, $storeCode: String = \"TJ\", $availability: String = \"1\", $published: String = \"1\") {\n  products(\n    filter: {store_code: {eq: $storeCode}, published: {eq: $published}, availability: {match: $availability}, category_id: {eq: $categoryId}}\n    currentPage: $currentPage\n    pageSize: $pageSize\n  ) {\n    items {\n      sku\n      item_title\n      category_hierarchy {\n        id\n        name\n        __typename\n      }\n      primary_image\n      primary_image_meta {\n        url\n        metadata\n        __typename\n      }\n      sales_size\n      sales_uom_description\n      price_range {\n        minimum_price {\n          final_price {\n            currency\n            value\n            __typename\n          }\n          __typename\n        }\n        __typename\n      }\n      retail_price\n      fun_tags\n      item_characteristics\n      __typename\n    }\n    total_count\n    pageInfo: page_info {\n      currentPage: current_page\n      totalPages: total_pages\n      __typename\n    }\n    aggregations {\n      attribute_code\n      label\n      count\n      options {\n        label\n        value\n        count\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n}\n"

In [3]:
category_request_info = [
        {
            "cateory": "Food",
            "variables": {"storeCode":"TJ","availability":"1","published":"1","categoryId":8,"currentPage":1,"pageSize":101},
            "page_size": 101
        },
        {
            "cateory": "Everything Else",
            "variables": {"storeCode":"TJ","availability":"1","published":"1","categoryId":215,"currentPage":1,"pageSize":9},
            "page_size": 9
        },
        {
            "cateory": "Beverages",
            "variables": {"storeCode":"TJ","availability":"1","published":"1","categoryId":182,"currentPage":1,"pageSize":19},
            "page_size": 19
        },
        {
            "cateory": "Flowers & Plants",
            "variables": {"storeCode":"TJ","availability":"1","published":"1","categoryId":203,"currentPage":1,"pageSize":2},
            "page_size": 2
        }
    ]

In [4]:
output = []
for info in category_request_info:
    for page in range(1, info['page_size']+1):
        info["variables"]['currentPage'] = page
        request_body = {
            "operationName": "SearchProducts",
            "query": query,
            "variables": info["variables"]
        }
        data = requests.post(url, json=request_body).json()
        for product in data['data']['products']['items']:
            temp = {}
            temp['brand'] = "Trader Joe's"
            temp['description'] = ''
            temp['image'] = 'https://www.traderjoes.com' + product['primary_image']
            temp['price'] = product['price_range']['minimum_price']['final_price']['value']
            temp['sale_price'] = product['retail_price']
            temp['title'] = product['item_title']
            temp['product_id'] = product['sku']
            temp['full_category'] = ' > '.join([sub_category['name'] for sub_category in product['category_hierarchy']])
           
            output.append(temp)
        time.sleep(random.randint(1,3))

In [5]:
output

[{'brand': "Trader Joe's",
  'description': '',
  'image': 'https://www.traderjoes.com/content/dam/trjo/products/m20907/50467.png',
  'price': 2.49,
  'sale_price': '2.49',
  'title': '"This Strawberry Walks into a Bar" Cereal Bars',
  'product_id': '050467',
  'full_category': 'Products > Food > For the Pantry > Cereals'},
 {'brand': "Trader Joe's",
  'description': '',
  'image': 'https://www.traderjoes.com/content/dam/trjo/products/m20203/78393.png',
  'price': 5.49,
  'sale_price': '5.49',
  'title': "Ivy's Reserve Somerset Red Cheese",
  'product_id': '078393',
  'full_category': 'Products > Food > Cheese > Wedges, Wheels, Loaves, Logs'},
 {'brand': "Trader Joe's",
  'description': '',
  'image': 'https://www.traderjoes.com/content/dam/trjo/products/m21001/77338.png',
  'price': 3.99,
  'sale_price': '3.99',
  'title': 'Savory Squares',
  'product_id': '077338',
  'full_category': 'Products > Food > Snacks & Sweets > Chips, Crackers & Crunchy Bites'},
 {'brand': "Trader Joe's",
  

In [8]:
with open("output/trader_joes.json", "w") as json_file:
    json.dump(output, json_file, indent=4)

In [31]:
class Validator:
    
    def __init__(self, data):
        self.data = data
        
    def validate_price(self):
        if float(data['sale_price']) <= float(data['price']):
            print("Price is available and correct.")
        else:
            l.error("Please add/check price.")
    
    def validate_mandatory_fields(self):
        if data['title'] != '':
            print("Title is available.")
        else:
            l.error("Title is mandatory. Please add title info in data.")
            
        if data['product_id'] != '':
            print("Product id is available.")
        else:
            l.error("Product id is mandatory. Please add product id in data.")
            
        if 'models' in data.keys() and data['models']['variants'][0]['id'] != '':
            print("Model id is available.")
        else:
            l.error("Model id is mandatory. Please add model id in data.")
            
    def validate_variants(self):
        if 'models' in data.keys() and data['models']['variants'][0]['image'] != '':
            print("Variant image is available.")
        else:
            l.error("Variant image is mandatory. Please add variant image url in data.")

In [32]:
data = output[0]

In [33]:
validator = Validator(data)
validator.validate_price()
validator.validate_mandatory_fields()
validator.validate_variants()

ERROR:root:Model id is mandatory. Please add model id in data.
ERROR:root:Variant image is mandatory. Please add variant image url in data.


Price is available and correct.
Title is available.
Product id is available.
